#### Was man hier noch tun sollte: 
1.) Vielleicht etwas tun, um zu verhindern, dass Paare direkt hintereinander zweimal violating sind. Das sollte theoretisch zumindest nicht so sein.
2.) K (die Matrix zum speichern bereits berechneter Kernel-Produkte) sparse machen
  

In [25]:
import numpy as np

In [26]:
#random test problem

d = 10
l = 100
data1x = np.random.randint(255, size=(d-1,l))
data1y = np.random.randint(255, size=(1,l))
data2x = np.random.randint(255, size =(d-1,l))
data2y = -np.random.randint(-20,255, size=(1,l))
data1 = np.concatenate((data1x,data1y))
data2 = np.concatenate((data2x,data2y))
data = np.concatenate((data1,data2), axis = 1)
data = np.transpose(data)

label1 = np.ones(l).astype(int)
label2 = - label1;
label = np.concatenate((label1,label2))

l = 2*l



In [27]:
def F(i,alpha):
    
    out = - label[i]
    for j in range(l):
        out += alpha[j]*label[j]*kernel(data[i],data[j])
    return out

### Maximal violating pair SMO

In [31]:
def smo_new(data,label,C,kernel,tol,violationcheckyesorno,kernel_identifier = None):
    
    
    
    def I_up_low_membership(alpha_i,label_i):
        
        #very important!
        null = 1e-16
        v = np.array([False,False])
        if (alpha_i < C-null and label_i == 1) or (alpha_i > null and label_i == -1):
            v[0] = True
        if (alpha_i < C-null and label_i == -1) or (alpha_i > null and label_i == 1):
            v[1] = True
        return v
        
        
    #data: images are rows of array, so the number of columns is 28**2 and the number of rows is the number of training data
    l = label.shape[0]
    alpha = np.zeros(l)
    
    I = np.zeros((2,l),dtype = bool)
                  
    #initialize I, now logical vector! I[0] = I_up, I[1] = I_low
    I[0] = label == 1
    I[1] = label == -1
    
    b_up = -1
    b_low = 1
    
    v = np.array(range(l))
    i_up_array = v[I[0]]
    i_low_array = v[I[1]]
    
    i_0 = i_up_array[0]
    j_0 = i_low_array[0]
    
    fcache = -label.astype(float)

    
    #iter = 0
    #cycle = 0
    
    #the recurrent computation of the of kernel pairs should be avoided, since it expensive,
    #especially when the kernel is other than the standard scalar product; hence, we cache all
    #Gram matrix rows once we computed them into the array K; we hope that this remains feasible
    #even for very large l, since hopefully only few rows of K will be filled
    
    #sparse, i.e., no memory overflow so far
    K = np.empty([l,l])
    
    rows_calc = []
    
    while(b_up < b_low - tol):
        
        #print(F(i_0,alpha) - F(j_0,alpha))
        
        alph1 = alpha[i_0]
        alph2 = alpha[j_0]
        
        y1 = label[i_0]
        y2 = label[j_0]
        
        F1 = fcache[i_0]
        F2 = fcache[j_0]
        
        s = y1*y2
                
        if s == -1:
            L = max(0,alph2-alph1)
            H = min(C,C+alph2-alph1)
        else:
            L = max(0,alph2+alph1-C)
            H = min(C,alph2+alph1)
        
        
        if i_0 not in rows_calc:
            if kernel_identifier == 'standard scalar product':
                K[i_0] = np.dot(data,data[i_0]) 
            else:
                K[i_0] = [kernel(data[i],data[i_0]) for i in range(l)]
            rows_calc.append(i_0)
        
        if j_0 not in rows_calc:
            if kernel_identifier == 'standard scalar product':
                K[j_0] = np.dot(data,data[j_0])
            else:
                K[j_0] = [kernel(data[i],data[j_0]) for i in range(l)]
            rows_calc.append(j_0)
            
        k11 = K[i_0,i_0]
        k12 = K[i_0,j_0]
        k22 = K[j_0,j_0]
        
        eta = 2*k12 - k11 - k22
        
        if eta < 0:
            a2 = alph2 - y2*(F1-F2)/eta
            if a2 < L:
                a2 = L
            elif a2 > H:
                a2 = H
        else:
            #print('Error: eta == 0')
            raise ValueError('Error: eta == 0')
        
        a1 = alph1 + s*(alph2-a2)
        
        
        #zum Vergleich, siehe unten, wo untersucht wird, wie oft es vorkommt, dass
        #ein Paar nach einem Schritt sofort wieder violating ist
        #alpha_old = np.empty(alpha.shape)
        #np.copyto(alpha_old, alpha) 
        
        fac_i_0 = y1 * (a1 - alph1)
        fac_j_0 = y2 * (a2 - alph2)
        fcache = fcache + fac_i_0 * K[i_0]  + fac_j_0 * K[j_0]
        
       
        
        # update alpha, I_up, I_low 
        alpha[i_0] = a1
        alpha[j_0] = a2
        
                 
        I[:,i_0] = I_up_low_membership(a1,y1)
        I[:,j_0] = I_up_low_membership(a2,y2)
        
        
        #now choose i_0,j_0 for next iteration and compute b_up, b_low for these i_0,j_0
     
        
        #i_0_old = i_0
        #j_0_old = j_0
               

        I_up = I[0]
        I_low = I[1]
        ind_up = v[I_up]
        ind_low = v[I_low]
        
        i_0s = np.argmin(fcache[ind_up])
        i_0 = ind_up[i_0s]
        b_up = fcache[i_0]
        
        j_0s = np.argmax(fcache[ind_low])
        j_0 = ind_low[j_0s]
        b_low = fcache[j_0]
        
    
        #if i_0_old == i_0 and j_0_old == j_0:
            #cycle += 1
            #print('Achtung, Paar zweimal hintereinander violating:')
            #print('i_0 =',i_0,'j_0 =',j_0)
            #print('alpha[i_0]_old =',alph1, 'alpha[j_0]_old =',alph2)
            #print('alpha[i_0] =',alpha[i_0], 'alpha[j_0] =',alpha[j_0])
            #print('fcache[i_0] =',fcache[i_0], 'fcache[j_0] =', fcache[j_0])
            #print('fcache[i_0]_old =',F(i_0,alpha_old), 'fcache[j_0]_old =', F(j_0,alpha_old))
            #print('b_up - b_low =', b_up - b_low)
            
        #iter += 1
        
        
    if violationcheckyesorno == 'yes':
              
        b_up = float('inf')
        b_low = -b_up
        for i in range(l):
            if I[0][i] == 1 and fcache[i] < b_up:
                b_up = fcache[i]
            if I[1][i] == 1 and fcache[i] > b_low:
                b_low = fcache[i]

        if b_low - tol <= b_up:
            violationstring = 'no tol-violation'
        else:
            violationstring = 'tol-violations!!'
    else:
        violationstring = 'no violation requested'    
   
    #print('Anzahl wiederholt dasselbe Paar =', cycle)
    #print('iter =', iter)
    
    return {'solution': alpha, 'violationcheck': violationstring}

In [29]:
def kernel(v1,v2):
    """standard scalar product"""
    return np.dot(v1,v2)

kernel_identifier = kernel.__doc__

In [30]:
%%time
result = smo_new(data,label,1,kernel,1e-3,'yes',kernel_identifier)
print(result)

{'solution': array([  0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         1.04405933e-03,   0.00000000e+00,   0.00000000e+00,
         0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         0.00000000e+00,   0.00000000e+00,   1.94407810e-02,
         0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         0.